
## Connexion a Hugging Face

In [ ]:
from huggingface_hub import login

login(token="token_hf")

## Exctracction des questions et reponses du document dans un fichier JSON


In [ ]:
import docx
import re
import json

def extract_text_from_docx(docx_path,output_txt="test.txt"):
    """Extrait le texte brut d'un fichier Word."""
    doc = docx.Document(docx_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(text)
    return text
def extract_text_from_pdf_no_out(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()

    return text
    
    return text
def parse_questions(text):
    """Parse uniquement les questions et options à partir du texte extrait."""
    questions = []
    question_blocks = re.split(r'Question \d+', text)
    
    
    with open("test_block.txt", "w", encoding="utf-8") as f:
        f.write("\n\n=== Nouveau Bloc ===\n\n".join(question_blocks))  
    
    for block in question_blocks[1:]:  # On saute la première partie (avant la première question)
        lines = block.strip().split('\n')
        question_text_lines = []
        options = {}
        current_option = None
        
        for line in lines:
            line = line.strip()
            if "Answer" in line:
                break  # Arrêter l'ajout de texte dès qu'on atteint "Answer"
            
            # Détecter une option (A., B., C., D.)
            match_option = re.match(r'([A-E])\.\s*(.*)', line)
            
            if match_option:
                current_option = match_option.group(1)
                options[current_option] = match_option.group(2).strip()
            elif current_option:
                options[current_option] += " " + line  # Ajouter le texte multi-ligne des options
            else:
                question_text_lines.append(line)
                
        question_text = ' '.join(question_text_lines).strip()
        
        if question_text and options:
            questions.append({
                "QuestionText": question_text,
                "Options": options
            })
    
    return questions


def parse_answers(text):
    """Parse uniquement les réponses correctes et explications à partir du texte extrait."""
    answers = []
    answer_blocks = re.split(r'Answer', text)[1:]  # Diviser après chaque 'Answer'
    
    with open("answer_test.txt", "w", encoding="utf-8") as f:
        f.write("\n\n=== Nouveau Bloc ===\n\n".join(answer_blocks))  
    
    for block in answer_blocks:
        lines = block.strip().split('\n')
        correct_answer = ""
        explanation_lines = []
        found_answer = False  # Pour savoir quand commencer à stocker l'explication
        
        for line in lines:
            line = line.strip()
            
            # Détection améliorée de la réponse correcte
            answer_match = re.search(r'The correct\s*answer\s*is\s*([A-E])', line, re.IGNORECASE)
            if answer_match:
                correct_answer = answer_match.group(1)
                found_answer = True  # Début de l'explication
                continue  # Passer à la ligne suivante directement
            
            # Ajout de l'explication (sans stopper trop tôt)
            if found_answer:
                explanation_lines.append(line)
        
        # Enregistrement de la réponse
        if correct_answer:
            answers.append({
                "CorrectAnswer": correct_answer,
                "Explanation": ' '.join(explanation_lines).strip()
            })
    
    return answers

def combine_questions_and_answers(questions, answers):
    """Combine les questions et réponses en une seule structure JSON avec le numéro de question."""
    combined = []
    for i, (q, a) in enumerate(zip(questions, answers), start=1):  # Démarre à 1
        combined.append({
            "QuestionNumber": i,  # 🔥 Ajout du numéro de la question
            "QuestionText": q["QuestionText"],
            "Options": q["Options"],
            "CorrectAnswer": a["CorrectAnswer"],
            "Explanation": a["Explanation"],
            "Question_type": "MCQ",
            "file":"Questions Sup OEB"
        })
    return combined

def save_to_json(data, output_path):
    """Ajoute les nouvelles données au fichier JSON existant sans l'écraser."""
    try:
        with open(output_path, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []
    
    existing_data.extend(data)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)
    
    print(f"Données ajoutées à {output_path}")

# Chemin du fichier Word
input_docx_path = "exam/Questions Sup OEB.docx"
output_json_path = "questions_and_answers.json"

# Exécuter le traitement
text = extract_text_from_pdf_no_out("test.txt")
questions_data = parse_questions(text)
answers_data = parse_answers(text)
combined_data = combine_questions_and_answers(questions_data, answers_data)
save_to_json(combined_data, output_json_path)

## Assignation des catégories pour chaque question


In [ ]:
import json
import docx
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def assign_questions_to_categories(questions_json, categories_docx, output_json):
    # Charger les questions
    with open(questions_json, "r", encoding="utf-8") as f:
        questions_data = json.load(f)
    
    # Charger les catégories depuis le fichier DOCX
    def load_categories(docx_path):
        doc = docx.Document(docx_path)
        categories = []
        current_category = None
        
        for para in doc.paragraphs:
            text = para.text.strip()
            if text and text[0].isdigit():  # Identifier une nouvelle catégorie
                current_category = text
            elif text and current_category:
                categories.append({"category": current_category, "description": text})
        
        return categories
    
    categories = load_categories(categories_docx)
    
    # Préparer les textes des catégories et questions
    category_texts = [cat["description"] for cat in categories]
    question_texts = [q["QuestionText"] for q in questions_data]
    
    # Transformer en vecteurs TF-IDF
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(category_texts + question_texts)
    
    # Séparer les vecteurs catégories et questions
    category_vectors = tfidf_matrix[: len(categories)]
    question_vectors = tfidf_matrix[len(categories) :]
    
    # Calculer la similarité cosinus
    similarity_matrix = cosine_similarity(question_vectors, category_vectors)
    
    # Assigner chaque question à la catégorie la plus proche
    for i, question in enumerate(questions_data):
        best_match_index = np.argmax(similarity_matrix[i])
        best_category = categories[best_match_index]["category"]
        question["Theme"] = best_category  # Ajout du thème dans la question
    
    # Sauvegarder les résultats avec les thèmes ajoutés
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(questions_data, f, indent=4, ensure_ascii=False)
    
    print(f"Fichier mis à jour avec les thèmes : {output_json}")
    

cat="Categories.docx"
question="fichier_questions.json"
output="output.json"
assign_questions_to_categories(question,cat,output)


## Supression des nombres pour insertion dans base de données

In [ ]:
import json
import re

def remove_numbers_from_theme(json_file, output_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    def clean_theme(theme):
        return re.sub(r'^\d+(\.\d+)*\s*', '', theme)
    
    if isinstance(data, list):
        for item in data:
            if "Theme" in item:
                item["Theme"] = clean_theme(item["Theme"])
    elif isinstance(data, dict) and "Theme" in data:
        data["Theme"] = clean_theme(data["Theme"])
    
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

# Exemple d'utilisation
input_file = "fichier.json"
output_file = "fichier.json"
remove_numbers_from_theme(input_file, output_file)


## Insertion des questions dans base de données firestore

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import json



def ajouter_questions_firestore(chemin_fichier_json, nom_collection):
    """
    Lit les données d'un fichier JSON et les ajoute à une collection Firestore.

    Args:
        chemin_fichier_json (str): Chemin vers le fichier JSON contenant les données.
        nom_collection (str): Nom de la collection Firestore.
    """
        # Vérifier si Firebase est déjà initialisé
    if not firebase_admin._apps:
        cred = credentials.Certificate("fichier_firebase.json")
        firebase_admin.initialize_app(cred)

    # Connexion à Firestore
    db = firestore.client()
    
    try:
        with open(chemin_fichier_json, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Vérifie que le JSON est bien une liste de dictionnaires
        if not isinstance(data, list):
            raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

        collection_ref = db.collection(nom_collection)

        for question in data:
            if not isinstance(question, dict):
                print(f"⚠️ Erreur : Un élément du JSON n'est pas un dictionnaire : {question}")
                continue  # Ignore les mauvais formats
            
            # Ajoute chaque question comme un document unique dans Firestore
            doc_ref = collection_ref.document(f"{question['QuestionNumber']}_{question['file']}")  # Utilisation du numéro de question comme ID
            doc_ref.set(question)

        print(f"✅ Données ajoutées avec succès à Firestore dans '{nom_collection}'.")

    except FileNotFoundError:
        print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' est introuvable.")
    except json.JSONDecodeError:
        print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' n'est pas un JSON valide.")
    except ValueError as ve:
        print(f"❌ Erreur de format JSON : {ve}")
    except Exception as e:
        print(f"❌ Une erreur s'est produite : {e}")

# Exemple d'utilisation
chemin_fichier_json = 'fichier.json'
nom_collection = 'qcm'

ajouter_questions_firestore(chemin_fichier_json, nom_collection)


## Insertion des thèmes dans base de données

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import json

# Vérifier si Firebase est déjà initialisé
if not firebase_admin._apps:
    cred = credentials.Certificate("fichier_firebase.json")
    firebase_admin.initialize_app(cred)

# Connexion à Firestore
db = firestore.client()

def ajouter_themes_firestore(json_data, nom_collection):
    """
    Envoie les thèmes et sous-thèmes dans Firestore.

    Args:
        json_data (dict): Données JSON à envoyer.
        nom_collection (str): Nom de la collection Firestore.
    """
    try:
        collection_ref = db.collection(nom_collection)

        for theme, sous_themes in json_data.items():
            doc_id = theme.replace(' ', '_')
            doc_ref = collection_ref.document(doc_id)

            # Préparer la structure avec les sous-thèmes sous forme de liste
            theme_data = {
                "theme": theme,
                "sous_themes": [
                    {"nom": sous_theme, "description": description}
                    for sous_theme, description in sous_themes.items()
                ]
            }

            # Créer un document unique pour chaque thème principal
            doc_ref.set(theme_data)

        print(f"✅ Données ajoutées avec succès à Firestore dans '{nom_collection}'.")

    except Exception as e:
        print(f"❌ Une erreur s'est produite : {e}")

# Charger les données JSON depuis le fichier
chemin_fichier_json = 'themes.json'
nom_collection = 'themes'

try:
    with open(chemin_fichier_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    ajouter_themes_firestore(data, nom_collection)

except FileNotFoundError:
    print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' est introuvable.")
except json.JSONDecodeError:
    print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' n'est pas un JSON valide.")
